https://arxiv.org/pdf/2312.16171 (prompts referenced from)

In [9]:
import cv2
import os
from datetime import datetime
from openai import OpenAI
import csv
import base64
from dotenv import load_dotenv
import pandas as pd
import time

In [10]:
def configure():
    load_dotenv
MODEL = "gpt-4o-mini"
client = OpenAI(api_key=os.getenv('api_key'))

In [11]:

def capture_and_save_frames(output_folder, max_frames=100):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(0)  # 0 is usually the default camera
    frame_count = 0
    saved_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        timestamp = time.time()
        timestamp_str = datetime.fromtimestamp(timestamp).strftime('%H_%M_%S')
        frame_filename = f"frame_{timestamp_str}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_path, frame)
        saved_count += 1
        print(f"Saved frame at {timestamp_str}")

        frame_count += 1

    cap.release()
    print(f"Total frames captured: {frame_count}")
    print(f"Frames saved: {saved_count}")
    return saved_count

def process_frames(frames_folder, output_csv, max_frames=100):
    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Timestamp', 'Description'])
        
        frames = sorted(os.listdir(frames_folder))
        
        for frame_filename in frames[:max_frames]:
            frame_path = os.path.join(frames_folder, frame_filename)
            timestamp_str = frame_filename.split('_')[1].replace('.jpg', '')
            
            description = process_image(frame_path)
            
            csvwriter.writerow([timestamp_str, description])
            
            print(f"Timestamp: {timestamp_str}")
            print(f"Description: {description}")
            print("---")
        
        if len(frames) > max_frames:
            print(f"Note: Only processed the first {max_frames} frames out of {len(frames)} total frames.")

def main(output_csv, max_frames=100):
    configure()
    frames_folder = "captured_frames"
    
    print(f"Capturing and saving {max_frames} frames from webcam...")
    capture_and_save_frames(frames_folder, max_frames)
    
    print(f"Processing the first {max_frames} frames with LLM...")
    process_frames(frames_folder, output_csv, max_frames)
    
    print(f"Processing complete. Results saved to {output_csv}")


In [12]:
output_csv = "./webcam_descriptions.csv"
max_frames_to_process = 100
main(output_csv, max_frames_to_process)

Capturing and saving ./video_descriptions.csv frames from webcam...


TypeError: '<' not supported between instances of 'int' and 'str'

In [5]:
def summarize_csv(csv_path):
    df = pd.read_csv(csv_path)
    
    combined_descriptions = " ".join(df['Description'].tolist())
    
    # Create a prompt for the LLM
    prompt = f"Summarize the following descriptions and give a new line after every fullstop, prevent using Another image or the words another moment, Prevent using similar words rather use synonyms, And finally tell me what happened in the video descriptions {combined_descriptions}"
    
    # Get the summary from the LLM
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text accurately and concisely."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
    )
    
    summary = response.choices[0].message.content.strip()
    
    print("Summary of CSV Descriptions:")
    print(summary)

    with open("summary.txt", "w") as f:
        f.write(summary)
    return summary

#Summary Exporter
csv_path = "./video_descriptions.csv"
summarize_csv(csv_path)

Summary of CSV Descriptions:
The video descriptions depict various scenes from a soccer match involving FC Barcelona and Real Madrid. 

A close-up showcases a player's legs in a Barcelona kit with golden cleats on a grassy field. 

A man in a suit walks past a player in a red and blue striped jersey. 

A soccer player in a red and blue striped jersey with the "Barcelona" logo walks on the field. 

Another player in a similar jersey looks up with a focused expression during the game. 

A young athlete in a blue and red striped jersey also displays a concentrated look. 

A player in a red and blue jersey with a yellow collar gazes upward with determination. 

The video captures a tense moment featuring Lionel Messi in a Barcelona jersey and Cristiano Ronaldo in a white jersey, with officials and fans in the background. 

Messi is seen walking past a referee while surrounded by spectators and security personnel. 

The footage highlights Messi interacting with a referee, with excited fans 

'The video descriptions depict various scenes from a soccer match involving FC Barcelona and Real Madrid. \n\nA close-up showcases a player\'s legs in a Barcelona kit with golden cleats on a grassy field. \n\nA man in a suit walks past a player in a red and blue striped jersey. \n\nA soccer player in a red and blue striped jersey with the "Barcelona" logo walks on the field. \n\nAnother player in a similar jersey looks up with a focused expression during the game. \n\nA young athlete in a blue and red striped jersey also displays a concentrated look. \n\nA player in a red and blue jersey with a yellow collar gazes upward with determination. \n\nThe video captures a tense moment featuring Lionel Messi in a Barcelona jersey and Cristiano Ronaldo in a white jersey, with officials and fans in the background. \n\nMessi is seen walking past a referee while surrounded by spectators and security personnel. \n\nThe footage highlights Messi interacting with a referee, with excited fans visible b